In [7]:
import os, sys
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logger.info("Application started successfully.")

2025-06-13 03:54:27,584 - INFO - Application started successfully.


In [2]:
# load environment variables

storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "main"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [3]:
# set pyspark configuration
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("Read_csv_from_azure_blob") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g")

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

25/06/13 03:53:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
spark.sql("DROP NAMESPACE IF EXISTS nessie.silver")
spark.sql("CREATE NAMESPACE nessie.silver")
spark.sql("SHOW NAMESPACES in nessie").show()


Py4JJavaError: An error occurred while calling o47.sql.
: org.apache.iceberg.exceptions.NamespaceNotEmptyException: Namespace 'silver' is not empty.
	at org.apache.iceberg.nessie.NessieIcebergClient.dropNamespace(NessieIcebergClient.java:338)
	at org.apache.iceberg.nessie.NessieCatalog.dropNamespace(NessieCatalog.java:316)
	at org.apache.iceberg.spark.SparkCatalog.dropNamespace(SparkCatalog.java:532)
	at org.apache.spark.sql.execution.datasources.v2.DropNamespaceExec.run(DropNamespaceExec.scala:42)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:49)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:107)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$w

In [ ]:
spark.sql(""" 
select *
from nessie.bronze.customer 
""").show()

+-----------+---------+--------+----------+-------------+------+------------+---------------+--------------+--------------+-------------+--------------------+---------------+-----------------+---------------+--------------------+--------------------+
|     UserId|FirstName|LastName| BirthDate|MaritalStatus|Gender|YearlyIncome|      Education|    Occupation|HouseOwnerFlag|TotalChildren|NumberChildrenAtHome|NumberCarsOwned|DateFirstPurchase|CommuteDistance|     source_filepath|  ingestion_datetime|
+-----------+---------+--------+----------+-------------+------+------------+---------------+--------------+--------------+-------------+--------------------+---------------+-----------------+---------------+--------------------+--------------------+
|      jon24|      Jon|    Yang|1971-10-06|            M|     M|    90000,00|      Bachelors|  Professional|             1|            2|                   0|              0|       2011-01-19|      1-2 Miles|abfss://datalake@...|2025-06-13T02:42:.

In [11]:
import notebooks.lib.silver_dim_adventureWorks_sql as sql

source_list = [
    {
        "table_name": "nessie.silver.product",
        "sql_create_table": sql.sql_create_silver_product,
        "sql_select": sql.sql_select_bronze_product,
        "enabled": True
    },
    {
        "table_name": "nessie.silver.customer",
        "sql_create_table": sql.sql_create_silver_customer,
        "sql_select": sql.sql_select_bronze_customer,
        "enabled": True
    }
]

In [12]:
import pyspark.sql.functions as SqlFunction

for item in source_list:
    table_name = item["table_name"]
    # create table
    logger.info(f'creating table if not exists:{table_name}')
    spark.sql(item["sql_create_table"])

    # select data from bronze table
    logger.info('selecting data from bronze')
    df = spark.sql(item["sql_select"])
    num_rows, num_columns = df.count(), len(df.columns)

    # insert data into silver
    logger.info(f'inserting data ({num_rows} rows {num_columns}, columns) {table_name}')
    df.writeTo(table_name).createOrReplace()
    logger.info(f'data inserted successfully')



2025-06-13 04:01:30,814 - INFO - creating table if not exists:nessie.silver.product
2025-06-13 04:01:31,703 - INFO - selecting data from bronze
2025-06-13 04:01:33,483 - INFO - inserting data (295 rows 9, columns) nessie.silver.product
2025-06-13 04:01:34,929 - INFO - data inserted successfully                     
2025-06-13 04:01:34,930 - INFO - creating table if not exists:nessie.silver.customer
2025-06-13 04:01:35,170 - INFO - selecting data from bronze
2025-06-13 04:01:35,372 - INFO - inserting data (18484 rows 17, columns) nessie.silver.customer
2025-06-13 04:01:36,190 - INFO - data inserted successfully


In [16]:
spark.sql(""" 
select *
from nessie.silver.product
""").show()

+----------+--------------------+------------------------+---------------------+--------------------+----------+----------------+--------------------+--------------------+
|ID_PRODUCT|        NAME_PRODUCT|NAME_PRODUCT_SUBCATEGORY|NAME_PRODUCT_CATEGORY|    TEXT_DESCRIPTION|NAME_COLOR|AMT_PRODUCT_COST|TEXT_SOURCE_FILEPATH|  DATETIME_INGESTION|
+----------+--------------------+------------------------+---------------------+--------------------+----------+----------------+--------------------+--------------------+
|   BB-7421|   LL Bottom Bracket|         Bottom Brackets|           Components|     Chromoly steel.|        NA|         23,9716|abfss://datalake@...|2025-06-13T02:42:...|
|   BB-8107|   ML Bottom Bracket|         Bottom Brackets|           Components|Aluminum alloy cu...|        NA|         44,9506|abfss://datalake@...|2025-06-13T02:42:...|
|   BB-9108|   HL Bottom Bracket|         Bottom Brackets|           Components|Aluminum alloy cu...|        NA|         53,9416|abfss://dat